# Data Exploration with Python and Jupyter

Basic usage of the Pandas library to download a dataset,
explore its contents, clean up missing or invalid data,
filter the data according to different criteria,
and plot visualizations of the data.

- [Part 1: Python and Jupyter](index.html)
- [Part 2: Pandas with toy data](pandas-toy-data.slides.html)
- **Part 3: Pandas with real data**

# Let's download some real data

For some reason, the London Fire Brigade provides a public spreadsheet of all animal rescue incidents since 2009:

https://data.london.gov.uk/dataset/animal-rescue-incidents-attended-by-lfb

They provide a link to the dataset in csv (comma-delimited) format

In [ ]:
# import the Pandas library

import pandas as pd

In [ ]:
# download a csv file with some data and convert it to a DataFrame
url = "https://data.london.gov.uk/download/animal-rescue-incidents-attended-by-lfb/8a7d91c2-9aec-4bde-937a-3998f4717cd8/Animal%20Rescue%20incidents%20attended%20by%20LFB%20from%20Jan%202009.csv"
df = pd.read_csv(url, encoding="unicode_escape")

# Sidenote

You may be wondering what this `encoding='unicode_escape'` parameter is for, and how did I know it needed to be there?

Well, I first tried `pd.read_csv(url)` and got this:

```
---------------------------------------------------------------------------
UnicodeDecodeError                        Traceback (most recent call last)
/tmp/ipykernel_514329/874975015.py in <module>
----> 1 pd.read_csv(url)

~/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pandas/io/parsers.py in read_csv(filepath_or_buffer, sep, delimiter, header, names, index_col, usecols, squeeze, prefix, mangle_dupe_cols, dtype, engine, converters, true_values, false_values, skipinitialspace, skiprows, skipfooter, nrows, na_values, keep_default_na, na_filter, verbose, skip_blank_lines, parse_dates, infer_datetime_format, keep_date_col, date_parser, dayfirst, cache_dates, iterator, chunksize, compression, thousands, decimal, lineterminator, quotechar, quoting, doublequote, escapechar, comment, encoding, dialect, error_bad_lines, warn_bad_lines, delim_whitespace, low_memory, memory_map, float_precision, storage_options)
    608     kwds.update(kwds_defaults)
    609 
--> 610     return _read(filepath_or_buffer, kwds)
    611 
    612 

~/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pandas/io/parsers.py in _read(filepath_or_buffer, kwds)
    460 
    461     # Create the parser.
--> 462     parser = TextFileReader(filepath_or_buffer, **kwds)
    463 
    464     if chunksize or iterator:

~/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pandas/io/parsers.py in __init__(self, f, engine, **kwds)
    817             self.options["has_index_names"] = kwds["has_index_names"]
    818 
--> 819         self._engine = self._make_engine(self.engine)
    820 
    821     def close(self):

~/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pandas/io/parsers.py in _make_engine(self, engine)
   1048             )
   1049         # error: Too many arguments for "ParserBase"
-> 1050         return mapping[engine](self.f, **self.options)  # type: ignore[call-arg]
   1051 
   1052     def _failover_to_python(self):

~/.pyenv/versions/3.9.4/lib/python3.9/site-packages/pandas/io/parsers.py in __init__(self, src, **kwds)
   1896 
   1897         try:
-> 1898             self._reader = parsers.TextReader(self.handles.handle, **kwds)
   1899         except Exception:
   1900             self.handles.close()

pandas/_libs/parsers.pyx in pandas._libs.parsers.TextReader.__cinit__()

pandas/_libs/parsers.pyx in pandas._libs.parsers.TextReader._get_header()

pandas/_libs/parsers.pyx in pandas._libs.parsers.TextReader._tokenize_rows()

pandas/_libs/parsers.pyx in pandas._libs.parsers.raise_parser_error()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa3 in position 105: invalid start byte
```

## Suggested workflow / philosophy

- if in doubt, try!
- if you get an error, copy & paste the last bit into google
  - don't be intimidated by the long and apparently nonsensical error messages
  - almost certainly someone else has had this exact problem
  - almost certainly the solution is waiting for you
- look for a stackoverflow answer with many up-votes
  - ignore the green tick, this just means the person asking the question liked the answer
  - typically an answer with many up-votes is a better option
  - more recent answers can also be better: sometimes a library has changed since an older answer was written

(For anyone who wasn't already doing this, that may be the most useful thing in this course)

# Display the DataFrame

In [ ]:
df

# Display the data type of each column

In [ ]:
df.dtypes

# Convert DateTimeOfCall to a date-time

In [ ]:
df["DateTimeOfCall"].head(10)

In [ ]:
df["DateTimeOfCall"] = pd.to_datetime(df["DateTimeOfCall"], dayfirst=True)

In [ ]:
df["DateTimeOfCall"].head(10)

# Use the datetime as the index

In [ ]:
df.set_index("DateTimeOfCall", inplace=True)

In [ ]:
df

In [ ]:
# can use datetime to select rows: here is jan 2021
df["2021-01-01":"2021-01-31"]["FinalDescription"]

In [ ]:
import matplotlib.pyplot as plt

# can resample the timeseries: sum by month
df.resample("M")["IncidentNumber"].count().plot(title="Monthly Calls")
plt.show()

In [ ]:
# or by day
df.resample("d")["IncidentNumber"].count().plot(title="Daily Calls")
plt.show()

# Count the unique entries in each column

In [ ]:
df.nunique().sort_values()

In [ ]:
# this column is always the same:
df["TypeOfIncident"].unique()

In [ ]:
# so we can drop it from our dataframe:
df.drop("TypeOfIncident", axis=1, inplace=True)

In [ ]:
df.groupby("AnimalGroupParent")["IncidentNumber"].count().sort_values().plot.barh(
    logx=True
)
plt.show()

## Plot location of calls on a map

In [ ]:
# drop missing longitude/latitude
df2 = df.dropna(subset=["Longitude", "Latitude"])
# also drop zero values
df2 = df2[df2["Latitude"] != 0]
# convert to geodataframe using geopandas
import geopandas

gdf = geopandas.GeoDataFrame(
    df2, geometry=geopandas.points_from_xy(df2["Longitude"], df2["Latitude"])
)
# set crs to WGS84 Latitude/Longitude
gdf = gdf.set_crs(epsg=4326)
gdf.head()

In [ ]:
f, ax = plt.subplots(figsize=(16, 16))
# get a map of london
import contextily as cx

# london = cx.Place("London", source=cx.providers.CartoDB.Positron)
# plot london map image
# london.plot(ax=ax)
# transform coordinates to match london image
gdf = gdf.to_crs(epsg=3857)
# plot location of calls involving deer
gdf[gdf["AnimalGroupParent"] == "Cow"].plot(
    ax=ax, color="black", alpha=0.3, label="Cow"
)
gdf[gdf["AnimalGroupParent"] == "Deer"].plot(
    ax=ax, color="red", alpha=0.3, label="Deer"
)
gdf[gdf["AnimalGroupParent"] == "Fox"].plot(ax=ax, color="blue", alpha=0.3, label="Fox")
plt.title("Call locations by animal")
plt.legend()
plt.axis("off")
plt.show()